In [1]:
import pandas as pd

1. Load data
2. Clean data (Million song subset)
3. Merge datasets
4. Save data

In [5]:
million_songs_df = pd.read_csv('../data/raw/2_million_song_subset_raw.csv')
hot100_df = pd.read_csv('../data/raw/1_hot100.csv')

million_songs_df.head(30)

,title;artist
0,b'Je Sais Que La Terre Est Plate';b'Rapha\xc3\...
1,b'On Efface';b'Julie Zenatti'
2,b'Howells Delight';b'The Baltimore Consort'
3,b'Martha Served';b'I Hate Sally'
4,b'Zip-A-Dee-Doo-Dah (Song of the South)';b'Orl...
5,b'Liquid Time (composition by John Goodsall)';...
6,b'Misery Path (From the Privilege of Evil)';b'...
7,b'Nuovi Re pt. I I (feat. Tek money - Lady Tam...
8,b'Halloween';b'Dead Kennedys'
9,b'Parto em terras distantes';b'Brigada Victor ...
